In [1]:
%pip install keras keras-nlp kagglehub
%pip install ipywidgets tqdm
!wget -O data/databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
--2025-04-08 23:37:01--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 108.159.15.21, 108.159.15.3, 108.159.15.97, ...
Connecting to huggingface.co (huggingface.co)|108.159.15.21|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1744139222&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDEzOTIyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4Mz

In [12]:
import os
import json

# set kaggle username & key
with open("kaggle/kaggle.json") as file:
    for line in file:
        credentials = json.loads(line)
        os.environ["KAGGLE_USERNAME"]   = credentials["username"]
        os.environ["KAGGLE_KEY"]        = credentials["key"]

# set keras backend to JAX
os.environ["KERAS_BACKEND"]                     = "jax"
# avoid memory fragmentation (JAX setting)
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]    = "1.00"

import keras
import keras_nlp
print(f"Backend for Keras : {keras.backend.backend()}")

import jax

import tensorflow as tf

Backend for Keras : jax


In [13]:
prompts = []
responses = []

with open("data/databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        prompt = f"Instruction:\n{features.get('instruction', '')}\nContext:\n{features.get('context', '')}"
        response = features.get("response", "")
        prompts.append(prompt)
        responses.append(response)

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices({
    "prompts": prompts,
    "responses": responses
})

In [5]:
# visualize available gemma3 presets
print(keras_nlp.models.Gemma3CausalLM.presets.keys())

jax.clear_caches()
gemma3 = keras_nlp.models.Gemma3CausalLM.from_preset("gemma3_1b", dtype = "bfloat16")
gemma3.summary()

dict_keys(['gemma3_1b', 'gemma3_instruct_1b', 'gemma3_4b_text', 'gemma3_instruct_4b_text', 'gemma3_12b_text', 'gemma3_instruct_12b_text', 'gemma3_27b_text', 'gemma3_instruct_27b_text', 'gemma3_4b', 'gemma3_instruct_4b', 'gemma3_12b', 'gemma3_instruct_12b', 'gemma3_27b', 'gemma3_instruct_27b'])


100%|██████████| 4.47M/4.47M [00:02<00:00, 1.91MB/s]
2025-04-08 23:37:17.198332: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1744135637.198690   16062 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 268 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


100%|██████████| 1.86G/1.86G [02:21<00:00, 14.2MB/s] 


Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │     999,885,952 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 999,885,952 (1.86 GB)

 Trainable params: 999,885,952 (1.86 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
gemma3.backbone.enable_lora(rank = 7)
gemma3.summary()

Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │   1,001,027,456 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,001,027,456 (1.86 GB)

 Trainable params: 1,141,504 (2.18 MB)

 Non-trainable params: 999,885,952 (1.86 GB)

In [ ]:
# training
gemma3.preprocessor.sequence_length = 64

optimizer = keras.optimizers.AdamW()
loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metrics = [keras.metrics.SparseCategoricalAccuracy()]

gemma3.compile(optimizer = optimizer, loss = loss, weighted_metrics = metrics)

gemma3.fit(tf_dataset, epochs = 10)

Epoch 1/10
7506/7506 ━━━━━━━━━━━━━━━━━━━━ 453s 55ms/step - loss: 1.2276 - sparse_categorical_accuracy: 0.5168
Epoch 2/10


/home/mms/miniconda3/envs/dl/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3292/7506 ━━━━━━━━━━━━━━━━━━━━ 4:14 60ms/step - loss: 1.2301 - sparse_categorical_accuracy: 0.5173

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x74be0f26df50>>
Traceback (most recent call last):
  File "/home/mms/miniconda3/envs/dl/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


3562/7506 ━━━━━━━━━━━━━━━━━━━━ 3:54 60ms/step - loss: 1.2297 - sparse_categorical_accuracy: 0.5172